In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
result = pd.read_csv('Auction_result.csv')

# 데이터 상위 일부 행 확인 및 데이터 구조와 내용 파악
train.head(5)
# 데이터 프레임의 정보 확인
train.info()

test.head(4)
# 데이터 하위 일부 행 확인
submission.tail(2)
# 작업 편의성을 위한 데이터 이름 변경
train = train.rename(columns={'point.y':'point_y', 'point.x' : 'point_x'})
test = test.rename(columns={'point.y':'point_y', 'point.x' : 'point_x'})
# 결측값, 문자열을 가지는 데이터 열 제거
train = train.drop(labels = ['Auction_key', 'Auction_class', 'Bid_class', 'Appraisal_company', 'Appraisal_date', 
                'First_auction_date', 'Final_auction_date', 'Final_result', 'Creditor',
                'addr_do', 'addr_si', 'addr_dong' , 'addr_li', 'addr_san','addr_bunji1',
                'addr_bunji2','addr_etc', 'Apartment_usage', 'Preserve_regist_date', 'Specific', 
                'Share_auction_YorN', 'road_name', 'road_bunji1', 'road_bunji2', 'Close_date', 
                'Close_result'], axis = 1)

train.head()

test = test.drop(labels = ['Auction_key', 'Auction_class', 'Bid_class', 'Appraisal_company', 'Appraisal_date', 
                'First_auction_date', 'Final_auction_date', 'Final_result', 'Creditor',
                'addr_do', 'addr_si', 'addr_dong' , 'addr_li', 'addr_san', 'addr_bunji1',
                'addr_bunji2','addr_etc', 'Apartment_usage', 'Preserve_regist_date', 'Specific', 
                'Share_auction_YorN', 'road_name', 'road_bunji1', 'road_bunji2', 'Close_date', 
                'Close_result'], axis = 1)

test.head()
# 예측에 필요한 종속변수 할당
original_y = train['Hammer_price']
original_y.head()
# 학습에 필요한 독립변수 할당
features = ['Claim_price', 'Auction_count', 'Auction_miscarriage_count',
       'Total_land_gross_area', 'Total_land_real_area',
       'Total_land_auction_area', 'Total_building_area',
       'Total_building_auction_area', 'Total_appraisal_price',
       'Minimum_sales_price', 'Total_floor', 'Current_floor', 'point_y',
       'point_x']

original_x = train[features]
original_x.head()
# train set으로 검증하기 위해 train과 valid set으로 분리
### test_size는 0.2일때가 디폴트, 0.3은 데이터가 많고 모델 튜닝 단계일때, 0.1은 데이터가 매우 적을 때 사용
### random_state는 난수 시드로 동일한 결과를 얻기 위한 이정표라고 생각하기
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(original_x, original_y, test_size=0.3, random_state=42)
# 종속변수와 독립변수의 결합
### axis는 데이터셋 결합 방향으로, 0은 행 방향, 1은 열 방향
xy_train = pd.concat(objs=[X_train, y_train], axis = 1)
# 회귀분석 식
formula = """
Hammer_price ~ Claim_price + Auction_count
       + Auction_miscarriage_count + Total_land_gross_area
       + Total_land_real_area + Total_land_auction_area
       + Total_building_area + Total_building_auction_area
       + Total_appraisal_price + Minimum_sales_price + Total_floor
       + Current_floor + point_x + point_y
"""
# 회귀 모델 데이터 학습 및 X_valid로 예측
import statsmodels.api as sm  
model = sm.OLS.from_formula(formula, data = xy_train)  
model = model.fit()  
predict = model.predict(X_valid)  
# RMSE는 큰 이상치를 잡아내는 효과적인 도구 
### 루트(평균 - 예상), 값이 많으면 내부에서 더하고 개수로 나눠줌
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_valid, predict)**0.5
RMSE
# 제출용 데이터 학습 및 정리
original_xy = pd.concat([original_x, original_y], axis=1)
original_xy.head()

model = sm.OLS.from_formula(formula, data = original_xy)  
model = model.fit()  
predict = model.predict(test)

predict.head()

submission['Hammer_price'] = predict
submission.head()

submission.to_csv('submission.csv', index=False)
# 종속변수 분포 확인
### 데이터 개수에 따라 bins 사용 기준 달라짐 bins 20~30(데이터 ~1000), 30~50(데이터 ~10000), 50~100(데이터 ~100000)
### bins="auto"도 가능
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.histplot(data=train, x="Hammer_price", bins=100)
plt.show()
# x축 Auction_count, y축 Hammer_price로 하는 막대 그래프
fig, ax = plt.subplots(figsize = (10,10))

x = train['Auction_count']
y = train['Hammer_price']

ax.set_xlabel('Auction_count')
ax.set_ylabel('Hammer_price')

ax.bar(x,y)
plt.show()
# 부산에서 7억 이상인 아파트 시군구 분포 확인
train_busan = train[train['addr_do'] == '부산']

fig, ax = plt.subplots(figsize = (10,5), dpi = 100)

x = train_busan[train_busan['Hammer_price']>700000000]['addr_si'].value_counts().index
y = train_busan[train_busan['Hammer_price']>700000000]['addr_si'].value_counts().values

ax.tick_params(axis = 'x', rotation=45)
ax.bar(x,y)
plt.show()
# 서울에서 30억 이상인 아파트 시군구 분포 확인
train_seoul = train[train['addr_do'] == '서울']

fig, ax = plt.subplots(figsize = (10,5), dpi = 100)

x = train_seoul[train_seoul['Hammer_price']>3000000000]['addr_si'].value_counts().index
y = train_seoul[train_seoul['Hammer_price']>3000000000]['addr_si'].value_counts().values

ax.tick_params(axis = 'x', rotation=45)
ax.bar(x,y)
plt.show()
# 거래 횟수와 가격의 관계 선그래프
fig, ax = plt.subplots(figsize = (7,5))

x_seq = result[result['Auction_key']==3]['Auction_seq']
y_min_price= result[result['Auction_key']==3]['Minimum_sales_price']
y_price= result[result['Auction_key']==3]['Appraisal_price']

ax.set_xlabel('Auction_seq')
ax.set_ylabel('Price')

ax.plot(x_seq,y_min_price)
ax.plot(x_seq,y_price)
ax.legend(['Minimum_sales_price','Appraisal_price'])
plt.show()

fig, ax = plt.subplots(figsize = (10,5))

x_seq = result[result['Auction_key']==5]['Auction_seq']
y_min_price= result[result['Auction_key']==5]['Minimum_sales_price']
y_price= result[result['Auction_key']==5]['Appraisal_price']

ax.set_xlabel('Auction_seq')
ax.set_ylabel('Price')
ax.set_xticks([1,2,3,4])

ax.plot(x_seq,y_min_price)
ax.plot(x_seq,y_price)
ax.legend(['Minimum_sales_price','Appraisal_price'])
plt.show()
# 산점도로 감정가와 최저매각가 관계 확인
fig, ax = plt.subplots(figsize = (10,5))

y_price_comparison = result['Appraisal_price']
y_minprice_comparison = result['Minimum_sales_price']

ax.set_xlabel('min_price_comparison')
ax.set_ylabel('price_comparisonice')

plt.xlim(0, 0.5e10)
plt.ylim(0, 0.8e10)

ax.scatter(y_minprice_comparison,y_price_comparison)
plt.show()
# 총 토지 경매 면적과 총 토지 실면적 관계 확인
## 경매 면적 증가할수록 실면적도 증가
### 다중공산성 확인됨, 다중공산성이란 비슷하거나 같은 역할을 하는 독립변수가 있으면 회귀 부넛ㄱ에서 계수 추정이 불안정해져서 제거 요소
fig, ax = plt.subplots(figsize = (10,5))

area = train[['Total_land_auction_area','Total_land_real_area']]

ax.scatter(x = area['Total_land_auction_area'], y = area['Total_land_real_area'])

plt.xlabel('Total_land_auction_area')
plt.ylabel('Total_land_real_area')

plt.xlim([0,250])
plt.ylim([0,250])

plt.show()
# 마지막 행 제외한 중복값 제거
## 경매는 여러번 진행되며 가격이 떨어지기 때문에 최종가격만 결과에 포함되도록 처리
clean_result = result.drop_duplicates(subset = 'Auction_key', keep = 'last')
clean_result
# train 데이터와 clean_result값 결합
### how에서 outer는 합집합, left, right은 위치한 데이터 기준으로 결합
### on에서 어떤 열 기준으로 결합할지 선택, 기준으로 삼지 않은 열은 2개가 생길수도 있음
train_result = train.merge(clean_result, how = 'left', on = 'Auction_key')
train_result.head(3)
# 연도 정보 추출
train_result['Auction_date'] = pd.to_datetime(train_result['Auction_date'], errors = 'ignore')
    
train_result['year'] = train_result['Auction_date'].dt.year
train_result['year']
# 범주형 데이터를 이진 변수로 생성
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(train_result[['Bid_class']])
onehot_data = ohe.transform(train_result[['Bid_class']])
onehot_frame = pd.DataFrame(onehot_data, columns = ohe.categories_[0])

train_result = pd.concat([train_result, onehot_frame], axis = 1)

train_result = train_result.drop(['Bid_class', '일괄'], axis=1)
train_result.head(3)
# 범주형 데이터를 숫자 변수로 생성
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train_result['Auction_results'])
train_result['Auction_results'] = le.transform(train_result['Auction_results'])
train_result['Auction_results'].value_counts()
# 결측값에 최빈값 할당
print('addr_bunji1의 결측값은', train_result['addr_bunji1'].isnull().sum(), '개 입니다.')
freq = train_result['addr_bunji1'].value_counts().index[0]
train_result['addr_bunji1'] = train_result['addr_bunji1'].fillna(freq)
print('addr_bunji1의 결측값은', train_result['addr_bunji1'].isnull().sum(), '개 입니다.')
# 상관계수 확인
### 1에 가까울수록 양의 선형 관계, -1에 가까울수록 음의 선형 관계, 0에 가까울수록 관계없음
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (16,12), dpi = 100, constrained_layout=True)
sns.heatmap(train_result.corr(), annot = True)
plt.show()
# 다중공산성 의심 변수 제거
highcorr_area = ['Total_land_real_area', 'Total_land_auction_area', 'Total_building_area', 'Total_building_auction_area']
train_result = train_result.drop(highcorr_area, axis =1)
# 종속변수와 상관계수 높았던 피처 저장하여 변수 저장
selected = ['Appraisal_price', 'Total_appraisal_price', 'Minimum_sales_price_x', 'Hammer_price']

selected_train = train_result[selected]
selected_train.head(3)

# year 피처 생성 및 날짜 피처 제거
train['Final_auction_date'] = pd.to_datetime(train['Final_auction_date'], errors = 'ignore')
train['year'] = train['Final_auction_date'].dt.year
date_col = ['Appraisal_date', 'First_auction_date', 'Final_auction_date', 'Preserve_regist_date', 'Close_date']
train = train.drop(date_col, axis= 1)

# 최빈값으로 결측값 보완
addr_freq = train['addr_bunji1'].value_counts().index[0]
road_freq = train['road_bunji1'].value_counts().index[0]
train['addr_bunji1'] = train['addr_bunji1'].fillna(addr_freq)
train['road_bunji1'] = train['road_bunji1'].fillna(road_freq)

# 결측값 많은 피처 제거
much_null = ['addr_li', 'addr_bunji2', 'Specific', 'road_bunji2']
train = train.drop(much_null, axis = 1)

# Hammer_price를 제외하고 상관계수가 높았던 피처 제거
highcorr_col = ['Total_land_real_area', 'Total_land_auction_area', 'Total_building_area', 'Total_building_auction_area']
train = train.drop(highcorr_col, axis =1)

result['result_year'] = pd.to_datetime(result['Auction_date'], errors = 'ignore').dt.year

result = result[result['result_year'] >= 2014]

need_merge = result[['Auction_key', 'Auction_results']].drop_duplicates(subset = 'Auction_key', keep = 'last')
need_merge = need_merge.reset_index(drop = True)
need_merge.head(5)

result.head(43)
# Auction_key 기준으로 그룹화
### 그룹화시 판단기준 1. valid점수가 낮아지는지 2. 분산 낮아지는지 3. 평균끼리 차이 명확해졌는지 4. 샘플 수 충분한지
appraisal_min = result.groupby('Auction_key')['Appraisal_price'].min()
need_merge['appraisal_min'] = appraisal_min.values
need_merge.head()

sales_min = result.groupby('Auction_key')['Minimum_sales_price'].min()
need_merge['sales_min'] = sales_min.values
need_merge.head()

max_seq = result.groupby('Auction_key')['Auction_seq'].max()
need_merge['max_seq'] = max_seq.values
need_merge.head(2)
# 결과가 유찰 횟수 결합
failed_auction = result[result['Auction_results'] == '유찰']
auction_count = failed_auction.groupby('Auction_key')['Auction_results'].count()
auction_result = need_merge.join(auction_count, on = 'Auction_key', how = 'left', lsuffix = '_left', rsuffix = '_right')
auction_result.head(5)
# 결측치 0으로 채우기
auction_result = auction_result.fillna(0)
auction_result['Auction_results_right'].isnull().sum()

# 첫 경매시 최저매각가격
first_price = result.groupby('Auction_key')['Minimum_sales_price'].max().values # 첫
# 마지막 경매시 최저매각가격
last_price = result.groupby('Auction_key')['Minimum_sales_price'].min().values # 최종

change_rate = (last_price - first_price) / first_price * 100  
auction_result['change_rate'] = change_rate  
auction_result.head(2)   

train_result = train.merge(auction_result, on = 'Auction_key', how = 'left')
train_result.head(2)
# 유찰 횟수별 낙찰가 추이 확인
import matplotlib.pyplot as plt

pivot = train_result.pivot_table(index = 'year', columns = 'Auction_results_right', values = 'Hammer_price',  aggfunc = 'sum')
pivot.plot(figsize = (8,6))
plt.show()
# 빈도값 추출
objects = ['Auction_class', 'Bid_class', 'Appraisal_company', 'Final_result', 'Creditor', 'addr_do', 'addr_si', 'addr_dong', 'addr_san', 'addr_etc', 'Apartment_usage', 'Share_auction_YorN', 'road_name', 'Close_result', 'Auction_results_left']

for col in objects:
    print(train_result[col].value_counts()[:3])
    print(len(train_result[col].value_counts()))
    print('--------------------------')

check_company = train_result['Appraisal_company'].str[:-2][:50]
print(check_company)
# 종속변수 평균으로 시군구별 낙찰가 그래프 확인
import matplotlib.pyplot as plt
​
plt.figure(figsize=(4, 8), dpi=100, constrained_layout=True)
addr_index = train_result.groupby('addr_si')['Hammer_price'].mean().sort_values().index
addr_value = train_result.groupby('addr_si')['Hammer_price'].mean().sort_values().values
​
plt.xlabel('Hammer_price')
plt.ylabel('addr_si')
ax = plt.barh(y=addr_index, width=addr_value)
plt.show()
# 평균낙찰가 높은 지역 피처 생성
high_landprice = train_result.groupby('addr_si')['Hammer_price'].mean().sort_values(ascending = False).index[:6]

train_result['low_high'] = train_result['addr_si'].apply(lambda x : 1 if x in high_landprice else 0)
train_result['low_high'].value_counts()
# 원핫인코딩 (이진변수로 변환)
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
onehot_bid = ohe.fit_transform(train_result[['Bid_class']])
onehot_frame = pd.DataFrame(onehot_bid, columns = ohe.categories_[0])
train_result = pd.concat([train_result, onehot_frame], axis = 1)

train_result = train_result.drop(['Bid_class', '일괄'], axis = 1)
train_result.head(2)
# 라벨 인코딩 (변수 숫자로 할당)
from sklearn.preprocessing import LabelEncoder

label_col = ['Auction_class', 'addr_do', 'addr_san', 'Share_auction_YorN', 'Auction_results_left', 'Apartment_usage']

for i in label_col:
    le = LabelEncoder()
    train_result[i] = le.fit_transform(train_result[i])
# 피처 제거
drop_list = ['Close_result', 'Final_result', 'addr_dong', 'addr_etc', 'road_name', 'Appraisal_company', 'Creditor', 'addr_si']

train_result = train_result.drop(drop_list, axis = 1)
#---------------한번 끝까지 진행--------------------
# year 피처 생성 및 날짜 피처 제거
train['Final_auction_date'] = pd.to_datetime(train['Final_auction_date'], errors = 'ignore')
train['year'] = train['Final_auction_date'].dt.year
date_col = ['Appraisal_date', 'First_auction_date', 'Final_auction_date', 'Preserve_regist_date', 'Close_date']
train = train.drop(date_col, axis= 1)

# 최빈값으로 결측값 보완
addr_freq = train['addr_bunji1'].value_counts().index[0]
road_freq = train['road_bunji1'].value_counts().index[0]
train['addr_bunji1'] = train['addr_bunji1'].fillna(addr_freq)
train['road_bunji1'] = train['road_bunji1'].fillna(road_freq)

# 결측값이 많은 피처 제거
much_null = ['addr_li', 'addr_bunji2', 'Specific', 'road_bunji2']
train = train.drop(much_null, axis = 1)

# Target을 제외하고 상관계수가 높았던 피처 제거
highcorr_col = ['Total_land_real_area', 'Total_land_auction_area', 'Total_building_area', 'Total_building_auction_area']
train = train.drop(highcorr_col, axis =1)

# 데이터 분석 과정에서 제거하기로 결정한 피처 제거
drop_list = ['Close_result', 'Final_result', 'addr_dong', 'addr_etc', 'road_name', 'Appraisal_company', 'Creditor', 'addr_si']
train = train.drop(drop_list, axis = 1)

# result_year 변수 생성 및 2014년 이후 데이터만 사용
result['result_year'] = pd.to_datetime(result['Auction_date'], errors = 'ignore').dt.year
result = result[result['result_year'] >= 2014]

# 마지막 데이터 추출
need_merge = result[['Auction_key', 'Auction_results']].drop_duplicates(subset = 'Auction_key', keep = 'last')
need_merge = need_merge.reset_index(drop = True)

# 감정가 최솟값 결합
appraisal_min = result.groupby('Auction_key')['Appraisal_price'].min()
need_merge['appraisal_min'] = appraisal_min.values

# 최저매각가격 최솟값 결합
sales_min = result.groupby('Auction_key')['Minimum_sales_price'].min()
need_merge['sales_min'] = sales_min.values

# 경매횟수의 최대값 가져오기
max_seq = result.groupby('Auction_key')['Auction_seq'].max()
need_merge['max_seq'] = max_seq.values

# 경매결과가 유찰인 횟수 가져오기
failed_auction = result[result['Auction_results'] == '유찰']
auction_count = failed_auction.groupby('Auction_key')['Auction_results'].count()
auction_result = need_merge.join(auction_count, on = 'Auction_key', how = 'left', lsuffix = '_left', rsuffix = '_right')

# auction_results_right 결측치 보완
null_indice = auction_result[auction_result['Auction_results_right'].isnull()].index
auction_result.loc[null_indice, 'Auction_results_right'] = 0

# 증감률 피처 생성
last_price = result.groupby('Auction_key')['Minimum_sales_price'].min().values
first_price = result.groupby('Auction_key')['Minimum_sales_price'].max().values
auction_result['change_rate'] = (last_price - first_price) / first_price * 100

# 데이터 결합
train_result = train.merge(auction_result, on = 'Auction_key', how = 'left')
train_result.head(2)

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
onehot_bid = ohe.fit_transform(train_result[['Bid_class']])
onehot_frame = pd.DataFrame(onehot_bid, columns = ohe.categories_[0])
train_result = pd.concat([train_result, onehot_frame], axis = 1)

train_result = train_result.drop(['Bid_class', '일괄'], axis = 1)
train_result.head(2)

from sklearn.preprocessing import LabelEncoder

label_col = ['Auction_class', 'addr_do', 'addr_san', 'Share_auction_YorN', 'Auction_results_left', 'Apartment_usage']

for label in label_col:
    le = LabelEncoder()
    train_result[label] = le.fit_transform(train_result[label])

from sklearn.model_selection import train_test_split

train_x = train_result.drop(['Hammer_price', 'Auction_key'], axis = 1)
train_y = train_result['Hammer_price']

x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

import gc

del need_merge
del auction_result
del train_result

gc.collect()

import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error

ols = sm.OLS(y_train, x_train)
results = ols.fit()
valid_pred = results.predict(x_valid)

check_mse = root_mean_squared_error(y_valid, valid_pred)
check_mse

results.summary()
#--------------------설계된 피처 조합/학습 XGB/LGBM으로 수행
# year 피처 생성 및 날짜 피처 제거
train['Final_auction_date'] = pd.to_datetime(train['Final_auction_date'], errors='ignore')
train['year'] = train['Final_auction_date'].dt.year

date_col = ['Appraisal_date', 'First_auction_date', 'Final_auction_date',
            'Preserve_regist_date', 'Close_date']
train = train.drop(date_col, axis=1)

# 최빈값으로 결측값 보완
addr_freq = train['addr_bunji1'].value_counts().index[0]
road_freq = train['road_bunji1'].value_counts().index[0]

train['addr_bunji1'] = train['addr_bunji1'].fillna(addr_freq)
train['road_bunji1'] = train['road_bunji1'].fillna(road_freq)

# 결측값 많은 피처 제거
much_null = ['addr_li', 'addr_bunji2', 'Specific', 'road_bunji2']
train = train.drop(much_null, axis=1)

# Target을 제외하고 상관계수가 높았던 피처 제거
highcorr_col = ['Total_land_real_area', 'Total_land_auction_area',
                'Total_building_area', 'Total_building_auction_area']
train = train.drop(highcorr_col, axis=1)

# 데이터 분석 과정에서 제거하기로 결정한 피처 제거
drop_list = ['Close_result', 'Final_result', 'addr_dong', 'addr_etc',
             'road_name', 'Appraisal_company', 'Creditor', 'addr_si']
train = train.drop(drop_list, axis=1)

# year 피처 생성 및 날짜 피처 제거
test['Final_auction_date'] = pd.to_datetime(test['Final_auction_date'], errors = 'ignore')
test['year'] = test['Final_auction_date'].dt.year
date_col = ['Appraisal_date', 'First_auction_date', 'Final_auction_date', 'Preserve_regist_date', 'Close_date']
test = test.drop(date_col, axis= 1)

# 최빈값으로 결측값 보완
addr_freq = train['addr_bunji1'].value_counts().index[0]
road_freq = train['road_bunji1'].value_counts().index[0]
test['addr_bunji1'] = test['addr_bunji1'].fillna(addr_freq)
test['road_bunji1'] = test['road_bunji1'].fillna(road_freq)

# 결측값 많은 피처 제거
much_null = ['addr_li', 'addr_bunji2', 'Specific', 'road_bunji2']
test = test.drop(much_null, axis = 1)

# Target을 제외하고 상관계수가 높았던 피처 제거
highcorr_col = ['Total_land_real_area', 'Total_land_auction_area', 'Total_building_area', 'Total_building_auction_area']
test = test.drop(highcorr_col, axis =1)

# 데이터 분석 과정에서 제거하기로 결정한 피처 제거
drop_list = ['Close_result', 'Final_result', 'addr_dong', 'addr_etc', 'road_name', 'Appraisal_company', 'Creditor', 'addr_si']
test = test.drop(drop_list, axis = 1)

# result_year 변수 생성 및 2014년 이후 데이터만 사용
result['result_year'] = pd.to_datetime(result['Auction_date'], errors = 'ignore').dt.year
result = result[result['result_year'] >= 2014]

# 마지막 데이터 추출
need_merge = result[['Auction_key', 'Auction_results']].drop_duplicates(subset = 'Auction_key', keep = 'last')
need_merge = need_merge.reset_index(drop = True)

# 감정가 최솟값 결합
appraisal_min = result.groupby('Auction_key')['Appraisal_price'].min()
need_merge['appraisal_min'] = appraisal_min.values

# 최저매각가격 최솟값 결합
sales_min = result.groupby('Auction_key')['Minimum_sales_price'].min()
need_merge['sales_min'] = sales_min.values

# 경매횟수의 최대값 가져오기
max_seq = result.groupby('Auction_key')['Auction_seq'].max()
need_merge['max_seq'] = max_seq.values

# 경매결과가 유찰인 횟수 가져오기
failed_auction = result[result['Auction_results'] == '유찰']
auction_count = failed_auction.groupby('Auction_key')['Auction_results'].count()
auction_result = need_merge.join(auction_count, on = 'Auction_key', how = 'left', lsuffix = '_left', rsuffix = '_right')

# auction_results_right 결측치 보완
null_indice = auction_result[auction_result['Auction_results_right'].isnull()].index
auction_result.loc[null_indice, 'Auction_results_right'] = 0

# 증감률 피처 생성
last_price = result.groupby('Auction_key')['Minimum_sales_price'].min().values
first_price = result.groupby('Auction_key')['Minimum_sales_price'].max().values
auction_result['change_rate'] = (last_price - first_price) / first_price * 100

# 데이터 결합
train_result = train.merge(auction_result, on = 'Auction_key', how = 'left')
test_result = test.merge(auction_result, on = 'Auction_key', how = 'left')
train_result.head(2)

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, handle_unknown = 'ignore')
onehot_train = ohe.fit_transform(train_result[['Bid_class']])
onehot_frame = pd.DataFrame(onehot_train, columns = ohe.categories_[0])
train_result = pd.concat([train_result, onehot_frame], axis = 1)

train_result = train_result.drop(['Bid_class', '일괄'], axis = 1)
train_result.head(2)

test_onehot = ohe.transform(test_result[['Bid_class']])
onehot_frame = pd.DataFrame(test_onehot, columns = ohe.categories_[0])
test_result = pd.concat([test_result, onehot_frame], axis = 1)

test_result = test_result.drop(['Bid_class', '일괄'], axis = 1)

from sklearn.preprocessing import LabelEncoder
import numpy as np

label_col = ['Auction_class', 'addr_do', 'addr_san', 'Share_auction_YorN', 'Auction_results_left', 'Apartment_usage']

for col in label_col:
    le = LabelEncoder()
    train_result[col] = le.fit_transform(train_result[col])
    
    for label in np.unique(test_result[col]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_result[col] = le.transform(test_result[col])

from sklearn.model_selection import train_test_split

train_x = train_result.drop(['Hammer_price', 'Auction_key'], axis = 1)
train_y = train_result['Hammer_price']

x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

from xgboost import XGBRegressor

xgb = XGBRegressor(random_state = 42)
xgb.fit(x_train, y_train)
valid_pred = xgb.predict(x_valid)

from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y_valid, valid_pred)

from sklearn.model_selection import GridSearchCV

xgb_grid = XGBRegressor(random_state = 42)

params = {'n_estimators': [15, 30],
          'max_depth': [3, 8]}

greedy_CV = GridSearchCV(xgb_grid, param_grid=params, cv = 2, n_jobs = -1)
greedy_CV.fit(x_train, y_train)

xgb_best_model = greedy_CV.best_estimator_
valid_pred = xgb_best_model.predict(x_valid)
root_mean_squared_error(y_valid, valid_pred)

